In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from sklearn.preprocessing import StandardScaler
from random import choice


In [2]:
global_hypha_info = pd.read_pickle("global_hypha_info.pick")
time_plate_info = pd.read_pickle("time_plate_infos.pick")
time_hypha_info = pd.read_pickle("time_hypha_infos.pick")

In [3]:
plates = time_plate_info['Plate'].unique()
ind_plate_table_time = {}
for plate in plates:
    ind_plate_table_time[plate]=time_plate_info.loc[time_plate_info['Plate']==plate].copy()

In [6]:
plates = time_plate_info['Plate'].unique()
ind_plate_table = {}
for plate in plates:
    ind_plate_table[plate]=time_plate_info.loc[time_plate_info['Plate']==plate].copy()
    

In [8]:
plates = global_hypha_info['Plate'].unique()
ind_hypha_table = {}
for plate in plates:
    table = global_hypha_info.loc[global_hypha_info['Plate']==plate].copy()
    table = table.set_index('hypha')
    ind_hypha_table[plate]=table

In [9]:
for plate in plates:
    table = ind_plate_table[plate]
    table['total_length_m'] = table['tot_length_study']/10**6
    table['speed'] = table['tot_length_study'].diff(5)/table['time_since_begin'].diff(5)
    table['mean_step'] = np.mean(table['time_since_begin'].diff(1))
    table['density'] = table['tot_length_study']/(table['area_sep_comp']+1)
    table['ring_density'] = table['tot_length_study'].diff(10)/(table['area_sep_comp'].diff(10)+1)
    table['log_length_study'] = np.log2(table['tot_length_study'].astype(float))
    table['local_growth_rate'] = table['log_length_study'].diff(5)/table['time_since_begin'].diff(5)*24
    table_hypha = ind_hypha_table[plate]

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
plates = time_hypha_info['Plate'].unique()
time_hypha_table = {}
for plate in plates: 
    time_hypha_table[plate]=global_hypha_info.loc[global_hypha_info['Plate']==plate].copy()
    select_time_plate = ind_plate_table[plate]
    last_time = np.max(select_time_plate.loc[select_time_plate['out_study']==0]['time_since_begin'])
    last_time2 = np.max(select_time_plate.loc[select_time_plate['local_growth_rate']>=0.5]['time_since_begin'])
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
    select_time = select_time.loc[select_time['speed'].between(40,400)]
    select_global = global_hypha_info.loc[global_hypha_info['Plate']==plate]
    select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
    select_time = select_time.loc[select_time['distance_final_pos']>200]
    select_time = select_time.loc[select_time['in_ROI']=='True']
    select_time = select_time.loc[select_time['time_since_begin_exp']<=last_time2]
    select_time['speed'] = select_time['speed'].astype(float)
    time_hypha_table[plate] = select_time

In [11]:
plate = 152
table = ind_hypha_table[plate]
table_time = time_hypha_table[plate]
select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
table['max_speed'] = table_time.groupby('end')['speed'].max()
table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
table['log_growth'] = np.log10((table['tot_growth_C']+1).astype(float))

table['growth_lapse'] = table_time.groupby('end')['speed'].count()
table['speed_at_init'] = speed_at_init.groupby('end')['speed'].max()
table = table.loc[table['max_speed'].notnull()]
table = table.loc[table['av_width_final'].between(0,15)]
# table = table.loc[table['num_branch']>=0]
table = table.loc[table['tot_growth_C']>=200]

# table['branch_frequ'] = table['num_branch']/(table['tot_growth_C']+1)
# table = table.loc[table['branch_frequ'].between(0,0.003)]


In [13]:
from pandas.plotting import scatter_matrix
scatter_matrix(table[['av_width_final','log_length','mean_speed2','max_speed','log_growth']].fillna(0), alpha=0.15, figsize=(6, 6), diagonal="kde",s=70);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
plate = 94
table = ind_hypha_table[plate]
table_time = time_hypha_table[plate]
select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
table['max_speed'] = table_time.groupby('end')['speed'].max()
table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
table['growth_lapse'] = table_time.groupby('end')['speed'].count()
table['speed_at_init'] = speed_at_init.groupby('end')['speed'].max()
table = table.loc[table['max_speed'].notnull()]
table = table.loc[table['av_width_final'].between(0,15)]
table = table.loc[table['num_branch']>=1]
table['branch_frequ'] = table['num_branch']/table['tot_length_C']
table = table.loc[table['branch_frequ']<0.003]
from sklearn.cluster import KMeans
df = table[['av_width_final','log_length','max_speed']].fillna(0)
features = ['av_width_final','log_length','max_speed']# Separating out the features
# features = ['log_length','mean_speed2']# Separating out the features

x = table.loc[:, features].values# Separating out the target
# y = table.loc[:,['target']].values# Standardizing the features
x = StandardScaler().fit_transform(x)
kmeans = KMeans(n_clusters=2).fit(x)
centroids = kmeans.cluster_centers_
c= kmeans.labels_.astype(float)

table['category'] = c
table['is_rh'] = (table['category']==table.groupby('category')['max_speed'].mean().idxmax()).astype(int)

In [28]:
select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
RH = table.loc[table['category']==table.groupby('category')['max_speed'].mean().idxmax()]
BAS = table.loc[table['category']==table.groupby('category')['max_speed'].mean().idxmin()]
rh = choice(RH.index)

1.0

In [216]:
table_time.groupby('end')['speed']

In [31]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
# ylabel = 'maximum speed $(\mu m.s^{-1})$'
xlabel = 'log length $(log(\mu m))$'

table.plot('branch_frequ','av_width_final',c='is_rh',kind='scatter',colormap='Dark2',alpha = 0.2,s=200,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_xlim(2.5,4.5)
# ax.set_ylim(50,400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'log length $(log(\\mu m))$')

In [25]:
10**2.85

707.945784384138

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
ylabel = 'maximum speed $(\mu m.s^{-1})$'
xlabel = 'log length $(log(\mu m))$'

table.plot('av_width_final','speed_at_init',c='category',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_xlim(2.5,4.5)
# ax.set_ylim(50,400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'log length $(log(\\mu m))$')

In [26]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
ylabel = 'maximum speed $(\mu m.h^{-1})$'
xlabel = 'init speed $(\mu m.h^{-1})$'

table.plot('speed_at_init','av_width_final',c='category',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_xlim(2.5,4.5)
# ax.set_ylim(50,400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'init speed $(\\mu m.h^{-1})$')

In [77]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
ylabel = 'maximum speed $(\mu m.s^{-1})$'
xlabel = 'log length $(log(\mu m))$'

table.plot('log_length','anastomoses',c='category',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
ax.set_ylim(-0.5,1.5)
ax.set_xlim(2.5,4.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(2.5, 4.5)

In [222]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'branch frequ $(\mu m^{-1})$'
xlabel = 'mean speed $(\mu m.s^{-1})$'
# xlabel = 'log length $(log(\mu m))$'
select = RH.loc[RH['branch_frequ']>0].copy()
select['branch_dist'] = 1/select['branch_frequ'].astype(float)
select=select.loc[select['branch_dist'].between(0,2000)]
select.plot('mean_speed','branch_dist',c='category',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
d = np.polyfit(select['mean_speed'].astype(float),select['branch_dist'].astype(float),1)
f = np.poly1d(d)
ax.plot(select['mean_speed'].astype(float),f(select['mean_speed']))
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_ylim(0,0.002)
# ax.set_xlim(2.5,4.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'mean speed $(\\mu m.s^{-1})$')

In [162]:
trajectory = select_time.loc[select_time['end']==rh]
trajectory.plot('time_since_emergence','speed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time_since_emergence'>

In [149]:
table

,av_width_final,tot_length_C,tot_growth_C,Plate,path_exp,folder_analysis,timestep_anastomosis,tot_length_pp,tot_growth_pp,timestep_stop_growth,...,out_of_ROI,max_speed,mean_speed2,log_length,growth_lapse,log_growth,speed_at_init,branch_frequ,category,is_rh
hypha,,,,,,,,,,,,,,,,,,,,,
336,14.2341,4102.81,1207.48,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,None,3951.99,1166.66,6,...,None,175.914415,124.593285,3.613187,3.0,3.082240,72.656641,NaN,0.0,1
728,12.1165,14932.4,14932.4,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,None,14725.4,38887.1,66,...,41,285.604338,265.984947,4.174157,11.0,4.174157,NaN,NaN,0.0,1
734,8.68565,5661.97,5253.62,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,8,5611.47,5217.65,9,...,None,252.560323,218.661878,3.753044,6.0,3.720541,167.893208,NaN,0.0,1
736,8.16468,7810.22,6337.36,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,9,7653.86,6234.75,11,...,None,277.737249,259.969026,3.892719,5.0,3.801977,NaN,NaN,0.0,1
740,7.42706,3515.89,2762.2,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,None,3490.92,2752.71,7,...,5,154.688616,154.283373,3.546159,2.0,3.441412,154.688616,NaN,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34483,6.08618,2742.95,2448.72,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,None,2591.67,2303.95,41,...,None,169.336257,169.336257,3.438376,1.0,3.389116,169.336257,NaN,1.0,0
34941,5.69935,514.076,383.313,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,38,489.079,373.82,39,...,None,60.352646,60.352646,2.711871,1.0,2.584685,60.352646,NaN,1.0,0
34976,8.14816,1085.35,364.916,152,Analysis_1636111758346200198_0_100_Version9/ex...,Analysis_1636111758346200198_0_100_Version9,40,1074.15,431.336,48,...,None,61.040569,61.040569,3.035970,1.0,2.563382,61.040569,NaN,1.0,0


In [148]:
num_columns = 4

plates = [76,94,808,91,88,792,785,799,80,26,785,152]
# plates = [152]
fig, axs = plt.subplots(((len(plates)+1)//num_columns), num_columns)
for i,plate in enumerate(plates):
    print(plate)
    table = ind_hypha_table[plate]
    table_time = time_hypha_table[plate]

    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table = ind_hypha_table[plate]
    table_time = time_hypha_table[plate]
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
    speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['log_growth'] = np.log10((table['tot_growth_C']+1).astype(float))

    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table['speed_at_init'] = speed_at_init.groupby('end')['speed'].max()
    table = table.loc[table['max_speed'].notnull()]
    table = table.loc[table['av_width_final'].between(0,15)]
#     table = table.loc[table['num_branch']>=0]
    table = table.loc[table['tot_growth_C']>=200]

    table['branch_frequ'] = table['num_branch']/(table['tot_growth_C']+1)
#     table = table.loc[table['branch_frequ'].between(0,0.003)]
    from sklearn.cluster import KMeans
    df = table[['av_width_final','log_length','max_speed']].fillna(0)
    features = ['av_width_final','log_length','max_speed']# Separating out the features
    # features = ['log_length','mean_speed2']# Separating out the features

    x = table.loc[:, features].values# Separating out the target
    # y = table.loc[:,['target']].values# Standardizing the features
    x = StandardScaler().fit_transform(x)
    kmeans = KMeans(n_clusters=2).fit(x)
    centroids = kmeans.cluster_centers_
    c= kmeans.labels_.astype(float)

    table['category'] = c
    table['is_rh'] = (table['category']==table.groupby('category')['max_speed'].mean().idxmax()).astype(int)
    ax = axs[i//num_columns,i%num_columns]
    xlabel = 'average width $(\mu m)$'
    ylabel = 'maximum speed $(\mu m.h^{-1})$'
#     xlabel = 'log length $(log(\mu m))$'

    table.plot('av_width_final','max_speed',c='is_rh',kind='scatter',colormap='Dark2',alpha = 1/len(table)*20,s=200,ax=ax,title = plate,colorbar = False)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_xlim(2.5,12)
    ax.set_ylim(50,400)

#     ax.set_ylim(50,400)
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

76
94
808
91
88
792
785
799
80
26
785
152


Text(0.04, 0.5, 'maximum speed $(\\mu m.h^{-1})$')

In [43]:
num_columns = 4

plates = [76,94,808,91,792,799,80,26,]
fig, axs = plt.subplots(((len(plates)+1)//num_columns), num_columns)
for i,plate in enumerate(plates):
    print(plate)
    table = ind_hypha_table[plate]
    table_time = time_hypha_table[plate]

    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table = ind_hypha_table[plate]
    table_time = time_hypha_table[plate]
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
    speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['log_growth'] = np.log10((table['tot_growth_C']+1).astype(float))

    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table['speed_at_init'] = speed_at_init.groupby('end')['speed'].max()
    table = table.loc[table['max_speed'].notnull()]
    table = table.loc[table['av_width_final'].between(0,15)]
    table = table.loc[table['num_branch']>=0]
    table = table.loc[table['tot_growth_C']>=200]

    table['branch_frequ'] = (table['num_branch']+1)/(table['tot_growth_C']+1)
    table = table.loc[table['branch_frequ'].between(0,0.003)]
    from sklearn.cluster import KMeans
    df = table[['av_width_final','log_length','max_speed']].fillna(0)
    features = ['av_width_final','log_length','max_speed']# Separating out the features
    # features = ['log_length','mean_speed2']# Separating out the features

    x = table.loc[:, features].values# Separating out the target
    # y = table.loc[:,['target']].values# Standardizing the features
    x = StandardScaler().fit_transform(x)
    kmeans = KMeans(n_clusters=2).fit(x)
    centroids = kmeans.cluster_centers_
    c= kmeans.labels_.astype(float)

    table['category'] = c
    table['is_rh'] = (table['category']==table.groupby('category')['max_speed'].mean().idxmax()).astype(int)

    RH = table.loc[table['is_rh']==1]
    BAS = table.loc[table['is_rh']==0]
    select = RH.loc[RH['branch_frequ']>0].copy()
    select = select.loc[select['num_branch']>=4]
    select['branch_dist'] = 1/select['branch_frequ'].astype(float)
    select=select.loc[select['branch_dist'].between(200,2000)]
    select=select.loc[select['mean_speed2'].between(200,320)]
#     select=select.loc[select['mean_speed2'].between(150,350)]
    x = select['mean_speed2'].astype(float)
    y = select['branch_dist'].astype(float) 
    d = np.polyfit(x,y,1,w=select['num_branch'].astype(float)**1)
    f = np.poly1d(d)
    x = np.array(x)
    x = x[:,np.newaxis]
    a, _, _, _ = np.linalg.lstsq(x, y,rcond=None)

    ax = axs[i//num_columns,i%num_columns]
    select.plot('mean_speed2','branch_dist',c='is_rh',kind='scatter',colormap='Dark2',alpha = 0.8,s=20,ax=ax,title = plate,colorbar = False)
    ax.plot(select['mean_speed2'].astype(float),f(select['mean_speed2']),label =f'y = {round(100*d[0])/100}x+{round(100*d[1])/100}')
#     ax.plot(x, a*x, 'r-',label=a)
    xlabel = 'average speed $(\mu m.h^{-1})$'
#     ylabel = 'maximum speed $(\mu m.s^{-1})$'
    ylabel = 'average branching distance$(\mu m)$'

#     table.plot('log_length','av_width_final',c='category',kind='scatter',colormap='Dark2',alpha = 0.15,s=100,ax=ax,title = plate,colorbar = False)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_xlim(200,320)
    ax.set_ylim(100,2000)
    ax.legend(fontsize = 10)

# #     ax.set_ylim(50,400)
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

76
94
808
91
792
799
80
26


Text(0.04, 0.5, 'average branching distance$(\\mu m)$')

In [108]:
x

array([[308.58646376],
       [325.5151731 ],
       [216.74382058],
       [333.29286752],
       [227.16606572],
       [234.0782224 ],
       [292.2086502 ],
       [305.06348057],
       [231.0226894 ],
       [295.15952704],
       [268.94994473],
       [292.37066381],
       [248.38115347],
       [283.20778144],
       [247.66994729],
       [286.11237057],
       [297.88951672],
       [327.78966029],
       [218.93234926],
       [194.2655037 ]])

In [93]:
d

array([  0.86913637, 410.65240685])

In [262]:
ind_hypha_table[152]['num_branch']

hypha
0         NaN
3         NaN
5         NaN
35        NaN
39        NaN
         ... 
310831    NaN
310843    NaN
310937    NaN
310973    NaN
311006    NaN
Name: num_branch, Length: 11272, dtype: object

In [307]:
scatter_matrix(table[['av_width_final','log_length','mean_speed2','max_speed']].fillna(0), alpha=0.1, figsize=(6, 6), diagonal="kde",);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
centroids

array([[  7.10461214,   2.87616725, 118.84248229],
       [  7.79537543,   3.22709095, 228.49994949]])

In [120]:
c= kmeans.labels_.astype(float)
c

array([1., 0., 0., ..., 1., 1., 1.])

In [111]:
features = ['av_width_final','log_length','mean_speed2']# Separating out the features
x = table.loc[:, features].values# Separating out the target
# y = table.loc[:,['target']].values# Standardizing the features
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df[['target']]], axis = 1)

In [113]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
ax.scatter(finalDf['principal component 1']
           , finalDf['principal component 2'],alpha = 0.2,s=20)


# ax.legend(targets)
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
pca.components_ 

array([[ 0.44901557,  0.6454717 ,  0.61786026],
       [ 0.88439657, -0.22246637, -0.4103065 ]])

In [75]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"# load dataset into Pandas DataFrame
df = pd.read_csv(url, names=['sepal length','sepal width','petal length','petal width','target'])

In [77]:
features = ['sepal length', 'sepal width', 'petal length', 'petal width']# Separating out the features
x = df.loc[:, features].values# Separating out the target
y = df.loc[:,['target']].values# Standardizing the features
x = StandardScaler().fit_transform(x)

In [82]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df[['target']]], axis = 1)

In [83]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['target'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
for plate in plates:
    table = ind_plate_table[plate]
    table['speed'] = table['tot_length_study'].diff(5)/table['time_since_begin'].diff(5)
    table['density'] = table['tot_length_study']/(table['area_sep_comp']+1)
    table['ring_density'] = table['tot_length_study'].diff(10)/(table['area_sep_comp'].diff(10)+1)
    table['log_length_study'] = np.log2(table['tot_length_study'].astype(float))
    table['local_growth_rate'] = table['log_length_study'].diff(5)/table['time_since_begin'].diff(5)*24

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [63]:
1/24**3/3/(np.pi*200**2)

16

In [46]:
plt.close('all')
x = 'time_since_begin'
y = 'ring_density'
logy =False
ylim = (0,4000)
# ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = 'ring density (mm.mm-2)'
num_columns = 4
fig, axs = plt.subplots((len(plates)//num_columns), num_columns)
for i,plate in enumerate(plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,label = plate,logy=logy,ylim=ylim,kind = kind)
        #,)
    if len(out_study)>0:
        out_study.plot(x,y, ax = ax,title = plate,color = 'red',logy=logy,ylim = ylim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)

# plt.tick_params(labelcolor="none", bottom=False, left=False)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})


# plt.xlabel("Common X-Axis")

# plt.ylabel("Common Y-Axis")
# plt.tight_layout(pad=0.)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'ring density (mm.mm-2)')